In [1]:
import sys
import talib as talib
import pandas as pd
import numpy as np
from pyecharts import options as opts
sys.path.append("..") 
from imp import reload
from src.ts_stock import TsStock
from src.chart import Chart
from src.strategy import FirstStrategy
from src.sina_stock import SinaStock


tsStock = TsStock()
sinaStock = SinaStock()
chart = Chart();


/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [13]:
sma_periods = [8,17,25,43,99,145,318,453]
keyword = '苏泊尔'
init_amount = 100000

stock_info = tsStock.get_stock_info(keyword)
stragety = FirstStrategy(tsStock, keyword , start_date="2010-01-01",sma_periods = sma_periods,sina_realtime=sinaStock)
df = stragety.simulate(start_date="20170101",
                           init_amount=init_amount,
                       max_add_count=4,
                       stop_price_factor=4
                       
                       )

/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [14]:
line = chart.get_line_ba(df,init_amount,stock_info)
line.render_notebook()


In [15]:

bs = {}
bs['buy'] = df.loc[df['action']=='buy',['trade_date','unit_price']]
bs['sell'] = df.loc[df['action'] == 'sell', ['trade_date','unit_price']]
df_stop_price = df.loc[df['unit_account']>0,['trade_date','stop_price']]
trade_dates = np.array(df.trade_date)
kdatas = np.array(df.loc[:,['open','close','low','high']])
vols = np.array(df['vol'])
df['bol_up'], df['bol_mi'], df['bol_lo'] = talib.BBANDS(df.close, 20, 2, 2)
sma_dic = {}
for period in sma_periods:
    name = "SMA" + str(period)
#     sma = talib.SMA(np.array(df.close),period)
#     df[name] =sma;
    sma_dic[name]={}
    sma_dic[name]['name'] = name;
    sma_dic[name]['datas'] =np.array(df[name]).round(2).tolist();


grid_chart= chart.get_grid(
    stock_info,
    trade_dates.tolist(),
    kdatas.tolist(),
    vols.tolist(),
    ma_dic=sma_dic,
    bs = bs, 
    df_stop_price=df_stop_price,
    bol=df[['trade_date','bol_up','bol_mi','bol_lo']] 
)



turnover_chart = chart.get_turnover_chart(df)

adosc_chart = chart.get_adosc_chart(df)

grid_chart.add(
    turnover_chart,
    grid_opts=opts.GridOpts(
        pos_left="6%", pos_right="8%", pos_top="68%", height="7%"
    ),
)
grid_chart.add(
    adosc_chart,
    grid_opts=opts.GridOpts(
        pos_left="6%", pos_right="8%", pos_top="75%", height="7%"
    ),
)

grid_chart.render_notebook()


In [16]:
df.tail(10)[['close','high','add_price','stop_price','unit','unit_price','R','P/R','ATR','signal','pre_buy_count']]

,close,high,add_price,stop_price,unit,unit_price,R,P/R,ATR,signal,pre_buy_count
trade_date,,,,,,,,,,,
2019-10-10,72.43,73.17,72.487101,67.197324,0.0,0.0,8.493339,0.296956,2.040669,,0.0
2019-10-11,73.31,74.60,72.489688,67.176631,0.0,0.0,8.493339,0.400566,2.045842,,0.0
2019-10-14,75.36,75.36,72.489771,67.175966,0.0,0.0,8.493339,0.641932,2.046008,pre_buy,1.0
2019-10-15,76.67,78.15,72.519651,69.726928,0.0,0.0,8.493339,0.796170,2.105768,pre_buy,2.0
2019-10-16,77.90,79.30,72.530135,70.793050,0.0,0.0,8.493339,0.940990,2.126737,pre_buy,3.0
2019-10-17,77.59,79.17,72.533600,70.765328,0.0,0.0,8.493339,0.904491,2.133668,pre_buy,4.0
2019-10-18,77.00,78.44,72.522327,70.855515,0.0,0.0,8.493339,0.835024,2.111121,pre_buy,5.0
2019-10-21,77.57,78.47,72.511905,70.938895,0.0,0.0,8.493339,0.902136,2.090276,,0.0
2019-10-22,77.56,78.20,72.512299,70.935739,0.0,0.0,8.493339,0.900958,2.091065,,0.0


In [6]:
df.to_excel('strategy_detail.xlsx')

In [7]:
df.tail(10).to_excel('strategy_detail.xlsx')

In [8]:
75.36 - 0.5 * 2.046


74.337